## S-109A Milestone 4
## Twitter Bot Detection Model Analysis

**Summer 2018**<br>
**Contributers:** Vivek Mishra and Karan Bhandarkar


In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import json
import numpy as np # imports a fast numerical programming library
import scipy as sp #imports stats functions, amongst other things
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.cm as cm #allows us easy access to colormaps
import matplotlib.pyplot as plt #sets up plotting under plt
import pandas as pd #lets us handle data as dataframes
#sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
from bs4 import BeautifulSoup
import re
#import unidecode
from pandas.plotting import scatter_matrix
from wordcloud import WordCloud, STOPWORDS

In [2]:
account_full_DF = pd.read_json("data/account_details.json", lines=True)
tweets_full_DF = pd.read_json("data/tweets.json", lines=True)

In [3]:
account_full_DF.columns

Index(['contributors_enabled', 'created_at', 'default_profile', 'default_profile_image', 'description', 'entities', 'favourites_count', 'follow_request_sent', 'followers_count', 'following', 'friends_count', 'geo_enabled', 'has_extended_profile', 'id', 'id_str', 'isBot', 'is_translation_enabled', 'is_translator', 'lang', 'listed_count', 'location', 'name', 'notifications', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_location', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'protected', 'screen_name', 'status', 'statuses_count', 'time_zone', 'translator_type', 'url', 'utc_offset', 'verified'],
      dtype='object')

In [4]:
tweets_full_DF.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'isBot', 'is_quote_status', 'lang', 'name', 'place', 'possibly_sensitive', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'retweet_count', 'retweeted',
       'source', 'truncated', 'user'],
      dtype='object')

In [5]:
accountDF = account_full_DF[['id','isBot','name','id_str','screen_name','location','description','url','following','followers_count','friends_count','listed_count','created_at','favourites_count','verified','statuses_count','lang','status','default_profile','default_profile_image','has_extended_profile']]
tweetsDF = tweets_full_DF[['name','isBot','full_text','lang','created_at','retweeted','entities','coordinates','retweet_count','favorite_count','place','geo','is_quote_status']]

In [6]:
accountDF

,id,isBot,name,id_str,screen_name,location,description,url,following,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile
0,21287212,1,FintechBot,21287212,FintechBot,"Sheffield, England",A little twitter bot for all the latest financ...,http://t.co/shU9OfwfD8,NaN,9150,992,1026,2009-02-19 10:03:21,259,False,113719,en,{'created_at': 'Fri Jul 27 23:13:27 +0000 2018...,False,False,False
1,829186945267834880,1,CATargetBot,829186945267834880,CATargetBot,"The Cloud, CA","2018 Federal/CA candidacy/committee filings, c...",https://t.co/XHsD8f9rGH,NaN,1431,1,44,2017-02-08 04:35:48,12,False,18981,en,{'created_at': 'Sat Jul 28 00:30:05 +0000 2018...,True,False,False
2,950406387632402433,1,Finance Feed,950406387632402432,InvestorNewsBot,United Kingdom,UK Investment Information from a variety of fe...,None,NaN,7,14,0,2018-01-08 16:38:35,0,False,7320,en,{'created_at': 'Fri Jul 27 16:01:05 +0000 2018...,True,False,False
3,407659112,1,Forex Trading Bots,407659112,FxTradingBot,Riga,#ForexTrading #ForexRobots,https://t.co/EQbbvN1nWT,NaN,30851,33827,494,2011-11-08 10:54:31,6,False,109179,en,{'created_at': 'Sat Jul 28 00:30:01 +0000 2018...,True,False,False
4,906021147015135232,1,PFD Bot,906021147015135232,pfdbot,"Anchorage, AK",Tweeting out the companies Alaskans own in the...,https://t.co/cum0F6SCOd,NaN,175,0,1,2017-09-08 05:07:29,1,False,25733,en,{'created_at': 'Sat Jul 28 00:30:28 +0000 2018...,True,False,False
5,964161357959651328,1,CoinMarketCal Bot,964161357959651328,cmcal_bot,,Beep Bop! I am the official CoinMarketCal bot....,https://t.co/qkBQGxkS5e,NaN,4328,2,93,2018-02-15 15:35:55,0,False,3466,en,{'created_at': 'Fri Jul 27 20:22:53 +0000 2018...,False,False,False
6,1203840834,1,Netflix Bot,1203840834,netflix_bot,Silicon Valley,Tweets new releases on Netflix Instant. Create...,http://t.co/i4spceQkYr,NaN,6695,1,0,2013-02-21 10:12:23,163,False,11551,en,{'created_at': 'Fri Jun 29 12:30:10 +0000 2018...,False,False,False
7,2216299843,1,olivia taters,2216299843,oliviataters,,ugh dad,None,NaN,6845,5280,293,2013-11-26 19:50:33,77275,False,50896,en,{'created_at': 'Fri Jul 27 22:50:04 +0000 2018...,True,False,False
8,1249693405,1,Dear Assistant,1249693405,DearAssistant,Twitter Bot,I am a Twitter bot and I'll try to answer your...,http://t.co/8P3zxTfCWF,NaN,4701,1,146,2013-03-07 17:56:44,11,False,5317,en,{'created_at': 'Mon Jun 20 02:56:26 +0000 2016...,False,False,False
9,2704554914,1,Reverse OCR,2704554914,reverseocr,,A bot that picks a word and then draws randoml...,http://t.co/r2HzjsqHTU,NaN,2286,1,228,2014-08-03 17:26:28,0,False,4657,en,{'created_at': 'Mon Oct 30 18:07:31 +0000 2017...,True,False,False


In [7]:
tweetsDF.head()

,name,isBot,full_text,lang,created_at,retweeted,entities,coordinates,retweet_count,favorite_count,place,geo,is_quote_status
0,FintechBot,1,What Is Scroll Cryptocurrency? https://t.co/dg...,en,2018-07-27 23:13:27,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,2,0,None,None,False
1,FintechBot,1,Coinbase Enables European Customers to Buy Vir...,en,2018-07-27 22:45:09,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,1,1,None,None,False
2,FintechBot,1,NHS Digital leading the protection of patient ...,en,2018-07-27 22:29:29,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,0,0,None,None,False
3,FintechBot,1,Top 3 Companies Using Blockchain in the Solar ...,en,2018-07-27 22:13:28,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,0,0,None,None,False
4,FintechBot,1,"VC Firms Are Betting on the Technology, Not on...",en,2018-07-27 21:13:29,False,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,2,0,None,None,False


In [8]:
## Merging two data frames on IsBot columns

tweets_with_accounts = pd.merge(tweetsDF, accountDF, left_on="name", right_on="screen_name", how="left")

In [9]:
print("TweetsDF count is {} and AccountDF count is {} and tweetsWithAccountDF count is {}".
      format(len(tweetsDF), len(accountDF), len(tweets_with_accounts)))


TweetsDF count is 10396 and AccountDF count is 100 and tweetsWithAccountDF count is 10396


In [12]:
## Splitting the data into training and test data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Removing NaN indexes from dataframe
tweets_with_accounts = tweets_with_accounts[tweets_with_accounts.followers_count.notnull()]


training_data, test_data = train_test_split(tweets_with_accounts, test_size=.2)



print(" Training data size is {} and test data size is {}".format(len(training_data), len(test_data)))



 Training data size is 8290 and test data size is 2073


In [13]:
num_features =  ["retweet_count", "favorite_count", "followers_count", "friends_count",
                 "listed_count", "favourites_count", "statuses_count"]

scaler = StandardScaler()
for feat in num_features:
    #print(training_data[feat].values)
    training_data[feat] = scaler.fit_transform(training_data[feat].values.reshape(-1,1))
    test_data[feat] = scaler.fit_transform(test_data[feat].values.reshape(-1,1))


In [14]:
X_train = training_data[["retweet_count", "favorite_count", "followers_count", "friends_count","listed_count", "favourites_count", "statuses_count"]]
y_train = training_data[["isBot_x"]]

X_test = test_data[["retweet_count", "favorite_count", "followers_count", "friends_count","listed_count", "favourites_count", "statuses_count"]]
y_test = test_data[["isBot_x"]]

In [15]:
print("X_train shape is {}, and y_train shape is {}".format(X_train.shape, y_train.shape))

X_train shape is (8290, 7), and y_train shape is (8290, 1)


###  We are trying starting with a Base Model of a simple Logistic Regression Model

In [16]:
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegressionCV

from sklearn.preprocessing import PolynomialFeatures


logit_model = LogisticRegression().fit(X_train, y_train)

logistic_model_score = logit_model.score(X_test, y_test)


In [17]:

print("Train set score: {0:4.4}%".format(logit_model.score(X_train, y_train)*100))
print("Test set score: {0:4.4}%".format(logit_model.score(X_test, y_test)*100))

Train set score: 90.3%
Test set score: 88.57%


In [18]:
logic_model_cv = LogisticRegressionCV(Cs=[1,10,100,1000,10000], cv=3, penalty='l2', 
                                       solver='newton-cg').fit(X_train,y_train)


print("Test set score with Cross Validation: {0:4.4}%".format(logic_model_cv.score(X_test, y_test)*100))


Test set score with Cross Validation: 88.86%


In [ ]:
X_train = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_train)
X_test = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_test)

logic_model_cv = LogisticRegressionCV(Cs=[1,10,100,1000,10000], cv=3, penalty='l2', 
                                       solver='newton-cg').fit(X_train,y_train)


print("Test set score with Polynomial Features and with Cross Validation: {0:4.4}%".format(logic_model_cv.score(X_test, y_test)*100))
